# Load Libraries

In [ ]:
# Import necessary libraries
import pandas as pd  # Library for data manipulation and analysis
import numpy as np  # Library for numerical computing
import glob  # Library for file handling
import re  # Library for regular expressions
from tqdm import tqdm  # Library for creating progress bars

import plotly.graph_objects as go  # Library for creating interactive plots

# Preprocessing
from sklearn import preprocessing  # Library for data preprocessing
import keras  # Deep learning library
from keras.models import Sequential  # Sequential model for stacking layers
from keras.layers.core import Dense, Dropout, Activation  # Layers for fully connected neural networks
from keras.layers import LSTM  # LSTM layer for sequence modeling
from keras.models import load_model  # Loading pre-trained models
import matplotlib.pyplot as plt  # Library for basic data visualization
import h5py  # Library for handling large datasets
import datetime  # Library for date and time operations
import tensorflow as tf  # Deep learning library



**For preprocessing and modeling**
* sklearn.preprocessing is used for data preprocessing tasks, such as scaling or encoding categorical variables.
* keras is a deep learning library that provides tools for building and training neural networks.
* Sequential is a type of model in Keras that allows stacking layers sequentially.
* Dense represents a fully connected layer in a neural network.
* Dropout is used for regularization by randomly setting input units to 0 during training to prevent overfitting.
* Activation defines the activation function to introduce non-linearity in the neural network.
* LSTM is a type of recurrent neural network layer used for sequence modeling.
* load_model is used to load pre-trained models.
* matplotlib.pyplot is imported as plt for basic data visualization.
* h5py is used for handling large datasets stored in the Hierarchical Data Format (HDF5).
* datetime is used for handling date and time information.
* tensorflow is imported as tf, which is a deep learning library used for various operations.

In [ ]:
import tensorflow as tf  # Import the TensorFlow library
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))  # Print the number of available GPUs

# tf.config.list_physical_devices('GPU') retrieves the list of physical devices (GPUs) available for TensorFlow to use.
# len(tf.config.list_physical_devices('GPU')) calculates the length of the list of GPUs.
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU'))) prints the number of available GPUs.

# Summary :
# This code is useful when you want to check the number of GPUs available for your TensorFlow environment. 
# GPUs are commonly used in deep learning for faster computations on large neural networks. 
# Knowing the number of available GPUs can help determine the hardware resources and parallel
# processing capabilities of your system when running TensorFlow-based models.

In [ ]:
# The code defines a function named plot that takes four parameters: df, x_feature_name,
# y_feature_name, and title. The function is designed to plot data from a dataframe
def plot(df,x_feature_name,y_feature_name,title):
    """
    This function takes two dataframes as input and plots the number of calls per day and per week.

    Args:
    daily_df (pandas.DataFrame): A dataframe containing daily call data.
    weekly_df (pandas.DataFrame): A dataframe containing weekly call data.

    Returns:
    None
    """

    # A new instance of the go.Figure() class from the plotly.graph_objects library is created. This will be used to create the plot
    fig = go.Figure()
    # Add a trace for daily calls
    # A trace is added to the figure using the go.Scatter() class from plotly.graph_objects. 
    # It specifies the x and y data for the plot, assigns a name to the trace, 
    # and sets the mode to display lines and markers.
    fig.add_trace(
        go.Scatter(
            x=df[x_feature_name],
            y=df[y_feature_name],
            name=y_feature_name,
            mode='lines+markers'
        ))

 

    # Update xaxis properties
    # The x-axis and y-axis titles are updated using the update_xaxes() and update_yaxes() methods of the figure object.
    fig.update_xaxes(title_text='Date')

    # Update yaxis properties
    fig.update_yaxes(title_text=y_feature_name)

    # Update title and height
    # The layout of the figure is updated using the update_layout() method. The title, height, and width of the plot are set.
    fig.update_layout(
        title=f'{title}',
        height=500,
        width=1200
    )

    # Show the plot
    # The plot is displayed using the show() method of the figure object.
    fig.show()

    # Write the plot to an HTML file
    # fig.write_html(f'Visualization/btc.html')
# Summary:
# The code defines a function plot() that takes in a dataframe (df), x-axis feature name (x_feature_name),
# y-axis feature name (y_feature_name), and a title. Inside the function, a plot is created using plotly.graph_objects. 
# The provided x and y data from the dataframe are added as a trace to the plot. 
# The x-axis and y-axis titles are updated, along with the plot title, height, and width. 
# The plot is then displayed using fig.show(). There is an optional commented-out line that suggests writing the plot to an HTML file.


In [ ]:
# The code defines a function named downsample_dataframe that takes in three parameters: df (the DataFrame to be downsampled), 
# downsampling_frequency (the desired frequency to downsample the DataFrame to), 
# and fill_method (the method to use for filling missing values).
# The function aims to downsample the DataFrame and fill any missing value
def downsample_dataframe(df, downsampling_frequency, fill_method='mean'):
  """
  Downsamples a DataFrame and fills missing values.

  Args:
    df: The DataFrame to downsample.
    downsampling_frequency: The frequency to downsample the DataFrame to.
    fill_method: The method to use to fill missing values.

  Returns:
    The downsampled DataFrame.
  """

  # Convert the Timestamp column to a datetime object.
  # This line converts the 'Timestamp' column in the DataFrame to a datetime object using the pd.to_datetime() function from the pandas library.
  df['Timestamp'] = pd.to_datetime(df['Timestamp'])

  # Set the index of the DataFrame to the Timestamp column.
  # This line sets the 'Timestamp' column as the index of the DataFrame using the set_index() method
  df = df.set_index('Timestamp')

  # Fill the missing values.
  # This block of code fills the missing values in the DataFrame based on the specified fill_method. 
  # If fill_method is set to 'mean', the missing values are filled with the mean value of each 
  # column using the fillna() method and df.mean(). If fill_method is set to 'median', 
  # he missing values are filled with the median value of each column using df.median(). 
  # If an invalid fill_method is provided, a ValueError is raised
  if fill_method == 'mean':
    df = df.fillna(df.mean())
  elif fill_method == 'median':
    df = df.fillna(df.median())
  else:
    raise ValueError('Invalid fill_method: {}'.format(fill_method))

  # Downsample the DataFrame.
  # This line downsamples the DataFrame to the specified downsampling_frequency using the resample() method with 
  # downsampling_frequency as the argument. The mean() method is then applied to calculate the mean 
  # value for each downsampled interval. The resulting downsampled DataFrame is returned as the output of the function.
  df = df.resample(downsampling_frequency).mean()
  return df

# Summary:
# The code defines a function downsample_dataframe() that takes in a DataFrame, downsampling frequency, 
# and fill method. The function converts the 'Timestamp' column to a datetime object, sets it as the index, 
# fills missing values based on the specified fill method (mean or median), and then downsamples the DataFrame 
# to the desired frequency using the mean value within each downsampled interval. 
# The resulting downsampled DataFrame is returned as the output of the function




In [ ]:
# These lines use the glob module to find all file paths that match the pattern 'GreenD_reduced_version_03/*.csv'. 
# The resulting file paths are stored in path_lists.
# Then, the file paths are sorted alphabetically and assigned to sorted_file_paths
path_lists = glob.glob('GreenD_reduced_version_03/'+'*.csv')
sorted_file_paths = sorted(path_lists)


# This code defines a function extract_digits that takes a string as input and 
# uses regular expressions (re.findall()) to extract digits from the string. 
# It returns the extracted digits as an integer if they exist, otherwise it returns 0.
# Then, the sorted() function is used with key=extract_digits to sort the file
# paths based on the digits extracted from each path. Finally, the sorted file paths are printed
def extract_digits(string):
    # Extract digits from the string using regular expression
    digits = re.findall(r'\d+', string)
    return int(digits[0]) if digits else 0

sorted_file_paths = sorted(path_lists, key=extract_digits)
print(sorted_file_paths)


# In this block of code, an empty list conct_list is created. Then, a loop is executed over each sorted
# file path using sorted_file_paths. Inside the loop, each CSV file is read using pd.read_csv(),
# and the 'Timestamp' column is converted to a datetime object using pd.to_datetime() w
# ith the utc=True parameter and unit='s' to indicate that the values are in Unix timestamp format. 
# The 'Timestamp' column is then formatted to a specific string format '%Y-%m-%d %H:%M:%S' 
# using the dt.strftime() function. The resulting dataframe is appended to the conct_list

conct_list = []
for path in tqdm(sorted_file_paths ,desc='processing'):
    
    data = pd.read_csv(path)
    data['Timestamp'] = pd.to_datetime(data['Timestamp'],utc=True,unit='s')#format='%Y-%m-%d %H-%M-%S')
    data['Timestamp'] = data['Timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
    conct_list.append(data)
    
# The !mkdir command creates a directory named 'Combined-Dataset-version-03' 
# if it doesn't already exist. Then, pd.concat() is used to concatenate all 
# the dataframes in conct_list along the rows, creating a single combined dataframe named df
!mkdir 'Combined-Dataset-version-03'
df = pd.concat(conct_list)


# Summary:
# The code reads multiple CSV files located in the 'GreenD_reduced_version_03/' directory. 
# The file paths are sorted in ascending order based on the digits extracted from each path using a custom function.
# Each file is then loaded into a dataframe, and the 'Timestamp' column is converted to a datetime object and formatted.
# The dataframes are appended to a list. A new directory named 'Combined-Dataset-version-03' is created,
# and all the dataframes in the list are concatenated into a single dataframe named df.

In [ ]:
df.sort_values(by='Timestamp',inplace=True)
# This line of code sorts the DataFrame df based on the values in the 'Timestamp' column in ascending order. 
# The sort_values() function is used to perform the sorting operation. The by parameter specifies 
# the column to sort by, which in this case is 'Timestamp'. The inplace=True parameter is
# used to modify the DataFrame df in place, meaning the changes are applied directly to the DataFrame instead of creating a new sorted DataFrame

In [ ]:
import matplotlib.pyplot as plt

# Customize the histogram plot
# This line creates a new figure object with a specific size of 8 inches by 6 inches. The figsize parameter is used to set the width and height of the figure in inches.
plt.figure(figsize=(8, 6))  # Set the figure size

# Plot the histogram
# This line plots a histogram using the plot.hist() method of the DataFrame df. It specifies the following parameters:

# bins=10: Sets the number of bins (bars) in the histogram to 10.
# edgecolor='black': Sets the color of the edges of the bars to black.
# alpha=0.7: Sets the transparency of the bars to 0.7 (partially transparent).
# color='steelblue': Sets the color of the bars to steel blue.
# linewidth=2: Sets the width of the bar edge lines to 2 pixels.
df.plot.hist(
    bins=10,                   # Number of bins
    edgecolor='black',         # Color of the edges of the bars
    alpha=0.7,                 # Transparency of the bars
    color='steelblue',         # Color of the bars
    linewidth=2,               # Width of the bar edge lines
)

# Customize the plot labels and title
plt.title('Histogram of Five Second Sampling Frequency Data')    # Set the title
plt.xlabel('Values')              # Set the x-axis label
plt.ylabel('Frequency')           # Set the y-axis label

# Show the plot
plt.savefig('5sec.png')

plt.show()

# Summary:
# The code creates a histogram plot using the plot.hist() method of the DataFrame df. 
# It customizes the plot by specifying the number of bins, edge color, transparency, 
# bar color, and line width. The plot is given a title, x-axis label, and y-axis label. 
# The resulting plot is saved as an image file named '5sec.png', and then it is displayed on the screen

In [ ]:

# Downsample the DataFrame to 1-hour frequency and fill missing values using mean imputation.
# df_1_hour = downsample_dataframe(df, '1H', fill_method='mean'): The downsample_dataframe() function is called to downsample the DataFrame df to a 1-hour frequency and fill missing values using mean imputation. The resulting downsampled DataFrame is assigned to the variable df_1_hour.
# df_1_hour.reset_index('Timestamp', inplace=True): The index of the DataFrame df_1_hour is reset, converting the 'Timestamp' column from the index back to a regular column. The reset_index() method is used, and the inplace=True parameter is set to modify the DataFrame in place.
# df_1_hour.fillna(method='ffill', inplace=True): Missing values in the DataFrame df_1_hour are filled using forward fill (ffill) method, which propagates the last observed value forward. The fillna() method is used, and the inplace=True parameter is set to modify the DataFrame in place

df_1_hour = downsample_dataframe(df, '1H', fill_method='mean')
df_1_hour.reset_index('Timestamp',inplace=True)
df_1_hour.fillna(method='ffill',inplace=True)


# df_daily = downsample_dataframe(df, '24H', fill_method='mean'): The downsample_dataframe() function is called to downsample the DataFrame df to a daily frequency (24 hours) and fill missing values using mean imputation. The resulting downsampled DataFrame is assigned to the variable df_daily.
# df_daily.reset_index('Timestamp', inplace=True): The index of the DataFrame df_daily is reset, converting the 'Timestamp' column from the index back to a regular column. The reset_index() method is used, and the inplace=True parameter is set to modify the DataFrame in place.
# df_daily.fillna(method='ffill', inplace=True): Missing values in the DataFrame df_daily are filled using forward fill (ffill) method, which propagates the last observed value forward. The fillna() method is used, and the inplace=True parameter is set to modify the DataFrame in place.
# Downsample the DataFrame to 1-hour frequency and fill missing values using mean imputation.
df_daily = downsample_dataframe(df, '24H', fill_method='mean')
df_daily.reset_index('Timestamp',inplace=True)
df_daily.fillna(method='ffill',inplace=True)

# Summary:
# The code performs downsampling of the original DataFrame df to different 
# frequencies (1-hour and daily) using the downsample_dataframe() function.
# Missing values are filled using mean imputation, and the resulting downsampled
# DataFrames (df_1_hour and df_daily) are modified in place to reset the index and fill missing values using forward fill.

In [ ]:
import matplotlib.pyplot as plt

# Customize the histogram plot
plt.figure(figsize=(8, 6))  # Set the figure size

# Plot the histogram
df_1_hour.plot.hist(
    bins=10,                   # Number of bins
    edgecolor='black',         # Color of the edges of the bars
    alpha=0.7,                 # Transparency of the bars
    color='steelblue',         # Color of the bars
    linewidth=2,               # Width of the bar edge lines
)

# Customize the plot labels and title
plt.title('Histogram of One Hour Sampling Frequency Data')    # Set the title
plt.xlabel('Values')              # Set the x-axis label
plt.ylabel('Frequency')           # Set the y-axis label

# Show the plot
plt.savefig('1-Hours.png')

plt.show()
# Summary:
# The code creates a histogram plot using the plot.hist() method of the DataFrame df_1_hour. 
# It customizes the plot by specifying the number of bins, edge color, transparency, 
# bar color, and line width. The plot is given a title, x-axis label, and y-axis label. 
# The resulting plot is saved as an image file named '1-Hours.png', and then it is displayed on the screen.

In [ ]:
# df_1_hour.dropna(inplace=True)

In [ ]:
# convert an array of values into a dataset matrix
# This code defines a function named create_dataset that takes a dataset and an optional parameter look_back as input. It returns two values as a tuple
def create_dataset(dataset, look_back=15):
    """_summary_

    Args:
        dataset (_type_): _description_
        look_back (int, optional): _description_. Defaults to 15.

    Returns:
        _type_: _description_
    """
    # These lines initialize empty lists dataX and dataY
    dataX, dataY = [], []
    # This line starts a loop that iterates over the range from 0 to len(dataset)-look_back-1.
    # It will loop over the indices of the dataset, excluding the last look_back + 1 elements.
    for i in range(len(dataset)-look_back-1):
        # These lines extract a subsequence of length look_back from the dataset starting from index i. 
        # It assigns the subsequence to variable a and appends it to the dataX list.
        # It also appends the element at index i + look_back from the dataset to the dataY list
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
        # print('Value of a : {}'.format(a))
        # print('Value of y : {}'.format(dataset[i + look_back, 0]))
        
    # This line converts dataX and dataY lists into NumPy arrays and returns them as the output of the functio
    return np.array(dataX), np.array(dataY)
# Summary:
# The create_dataset function takes a dataset as input and creates input-output pairs for a sequence prediction problem. 
# It uses a sliding window approach to extract subsequences of length look_back from the dataset and assigns
# them as input sequences (dataX). The element following each input sequence is assigned as the corresponding output value (dataY). 
# The function returns the input and output sequences as NumPy arrays.

In [ ]:
data_df = df_1_hour

In [ ]:
plot(data_df,x_feature_name='Timestamp',y_feature_name='Summe',title='Sampling Frequency 1 Hours')



In [ ]:
plot(df_daily,x_feature_name='Timestamp',y_feature_name='Summe',title='Sampling Frequency Daily')



In [ ]:
import numpy as np
import math
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing



data_df = df_1_hour # Your data dataframe
# These lines perform feature scaling on the 'Summe' column of the DataFrame data_df 
# using Min-Max scaling. It creates an instance of MinMaxScaler and fits it to the data, 
# transforming the values to the range [0, 1]. The scaled data is assigned to the variable dataset
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
dataset = min_max_scaler.fit_transform(data_df['Summe'].values.reshape(-1, 1))

# split into train, validation, and test sets
# These lines split the dataset into train, validation, and test sets based on specified proportions.
# It calculates the sizes of each set and assigns the corresponding subsets of dataset to train_data, val_data, and test_data
train_size = int(len(dataset) * 0.7)
val_size = int(len(dataset) * 0.2)
test_size = len(dataset) - train_size - val_size

train_data = dataset[:train_size]
val_data = dataset[train_size:train_size+val_size]
test_data = dataset[train_size+val_size:]

# create train, validation, and test datasets
# These lines call the create_dataset function to create input-output pairs for the train, validation, 
# and test sets. The function takes the respective subsets of data (train_data, val_data, test_data) 
# and a look_back parameter, and returns input (x) and output (y) sequences. 
# The input-output pairs are assigned to x_train, y_train, x_val, y_val, x_test, and y_test
look_back = 20
x_train, y_train = create_dataset(train_data, look_back)
x_val, y_val = create_dataset(val_data, look_back)
x_test, y_test = create_dataset(test_data, look_back)


# reshape the input data
# These lines reshape the input sequences (x_train, x_val, x_test) to match the required input shape for the LSTM model. 
# The new shape is (batch_size, timesteps, features), where batch_size is the number of samples,
# timesteps is the number of time steps, and features is the number of features.
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_val = np.reshape(x_val, (x_val.shape[0], 1, x_val.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

# print the sizes of the datasets
print('Training Data Size:', train_data.shape)
print('Validation Data Size:', val_data.shape)
print('Testing Data Size:', test_data.shape)

# Summary:
# The code performs the following steps:

# Assigns a DataFrame to data_df.
# Performs Min-Max scaling on a specific column of the DataFrame.
# Splits the scaled dataset into train, validation, and test sets.
# Calls the create_dataset function to create input-output pairs for the train, validation, and test sets.
# Reshapes the input sequences to match the required shape for the LSTM model.
# Prints the sizes of the train, validation, and test datasets.


# Model Building | Fitting

In [ ]:

# Check if GPU is available
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU is available.")
    print("List of physical GPUs:")
    for device in physical_devices:
        print(device)
else:
    print("GPU is not available. Using CPU instead.")
    
    
# Summary:
# The code checks if a GPU is available and sets the memory growth option if it is.
# It then prints the availability and a list of physical GPUs if present
# . If no GPU is available, it prints a message indicating the use of the CPU instead.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

# Set the model parameters
# These lines define the model parameters, including the number of epochs, batch size, number of LSTM layers, and the input window size.
num_epochs = 100
batch_size = 16
num_layers = 4
input_window=20

# Create the LSTM model
# These lines define the architecture of the LSTM model using the Sequential API.
# The model consists of multiple LSTM layers with different configurations,
# followed by a Dense layer. The input shape of the model is defined based on the look_back value.
model = Sequential()
model.add(LSTM(input_window, input_shape=(1, look_back), return_sequences=True))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(30, return_sequences=True))
model.add(LSTM(5, return_sequences=True))
model.add(Dense(1))

# Compile the model
# This line compiles the model by specifying the loss function and optimizer to use during training.
model.compile(loss='mean_absolute_error', optimizer='adam')

# Get the model's parameters
# This line calculates the total number of trainable parameters in the model and assigns it to the model_params variable.
model_params = model.count_params()

# Create the model name based on parameters and training settings
# This line creates a string that represents the model name based on the model's parameters and training settings.
model_name = f"model_{num_epochs}_epochs_{batch_size}_batch_{num_layers}_layers_{model_params}_params_input_window_{input_window}"


# Define early stopping callback
# This line defines an EarlyStopping callback that monitors the validation loss and stops training if the loss does not improve for a specified number of epochs.
early_stopping = EarlyStopping(patience=10,
                               monitor = 'val_loss', 
                               restore_best_weights=True)

# Define TensorBoard callback with model-specific log folder
# This line defines a TensorBoard callback that logs training information and metrics to a specified directory.
tb_callback = TensorBoard(log_dir=f'logs/{model_name}/', write_graph=True, update_freq='epoch')

# Define model checkpoint callback with model-specific filename
# This line defines a ModelCheckpoint callback that saves the model's weights to a file after each epoch, only saving the best weights based on the validation loss.
checkpoint_callback = ModelCheckpoint(f'checkpoints/{model_name}_{{epoch:02d}}.h5', save_weights_only=True, save_best_only=True)

# Train the model with early stopping, checkpoints, and TensorBoard
# This line trains the model using the fit function, providing the training and validation data, 
# number of epochs, batch size, verbosity level, and the defined callbacks for early stopping,
# TensorBoard, and model checkpointing. The training history is stored in the history variable
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), 
                    epochs=num_epochs, 
                    batch_size=batch_size, 
                    verbose=2,
                    callbacks=[early_stopping, tb_callback, checkpoint_callback])

# Print model summary
# This line prints a summary of the model's architecture and the number of trainable parameters.
print(model.summary())

# Summary:
# The code builds and trains an LSTM model with specified parameters and settings. It defines the model architecture, 
# compiles the model, sets up callbacks for early stopping, TensorBoard, and model checkpointing, 
# and trains the model using the provided data. Finally, it prints a summary of the model's architecture and parameters.

# Saving the Model 

In [ ]:
try:
    import joblib
except:
    !pip install joblib
    import joblib
    # Save the model
joblib.dump(model, 'model.pkl')
# Load the model
loaded_model = joblib.load('model.pkl')



In [1]:
import numpy as np
import pickle
import os

def load_model_and_scaler(model_path=None, scaler_path=None):
    # Use default paths if not provided
    model_path = model_path# or MODEL_PATH
    scaler_path = scaler_path #or SCALER_PATH

    # Load the model from the .pkl file
    with open(model_path, 'rb') as file:
        loaded_model = pickle.load(file)

    # Load the scaler from the .pkl file
    with open(scaler_path, 'rb') as file:
        scaler = pickle.load(file)

    return loaded_model, scaler


def predict_next_points(model_path, scaler_path, input_data, num_predictions=24):
    """
    Predicts the next points in a sequence using the provided model.

    Args:
        model_path (str): Path to the saved model file (.pkl).
        scaler_path (str): Path to the saved scaler file (.pkl).
        input_data (ndarray): Input sequence of points.
        num_predictions (int): Number of points to predict. Default is 24.

    Returns:
        ndarray: Predicted sequence of points.
    """

    # Load the model from the .pkl file
    with open(model_path, 'rb') as file:
        loaded_model = pickle.load(file)

    # Load the scaler from the .pkl file
    with open(scaler_path, 'rb') as file:
        scaler = pickle.load(file)

    # Reshape the input data
    input_data = np.array(input_data).reshape(1, 1, -1)

    # Predict the next points iteratively
    predictions = []
    for _ in range(num_predictions):
        # Predict the next point
        next_point = loaded_model.predict(input_data)[:, -1, 0]
        predictions.append(next_point)

        # Append the predicted point to the input data
        input_data = np.append(input_data[:, :, 1:], next_point.reshape(1, 1, 1), axis=2)

    # Rescale the predicted values
    rescaled_predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

    # Return the rescaled predicted sequence of points
    return rescaled_predictions


# Example usage

# Assuming you have a trained model and a scaler object
model = model
scaler = min_max_scaler

# Assuming you have the input data
input_data = np.array([1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 3, 4, 5, 6, 7, 8, 9, 20, 21])

# Saving the model and scaler
folder_name = 'model_files'
save_model_and_scaler(model, scaler)

# Predicting the next points and rescaling
predicted = predict_next_points(
    os.path.join(folder_name, 'model.pkl'),
    os.path.join(folder_name, 'scaler.pkl'),
    input_data
)

# Printing the rescaled predictions
print(predicted)


NameError: name 'model' is not defined

In [38]:
predicted

array([[29.546246],
       [29.380224],
       [29.351662],
       [29.412067],
       [29.441767],
       [29.547377],
       [28.734745],
       [28.565233],
       [29.227211],
       [29.209047],
       [29.116936],
       [28.954638],
       [29.42141 ],
       [28.636845],
       [29.00268 ],
       [28.994223],
       [29.13886 ],
       [29.127382],
       [29.102297],
       [28.583063],
       [27.912226],
       [27.911564],
       [27.911356],
       [27.911121]], dtype=float32)

In [ ]:
loaded_model = joblib.load('model.pkl')
loaded_model.summary()

In [ ]:
import matplotlib.pyplot as plt

# Get the training and validation loss values from the history object
# These lines extract the training loss and validation loss values from the history object, which was obtained during the model training.
train_loss = history.history['loss']
val_loss = history.history['val_loss']
 
# Plot the training loss and validation loss
# These lines create line plots for the training loss and validation loss values. The plot function is used to plot the values on a graph.
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
# These lines set the labels for the x-axis and y-axis, title of the plot, and enable the legend to display the labels of the plotted lines.
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()
# Summary:
# The code visualizes the training and validation loss values obtained during the model training. It plots the training loss and validation loss on the same graph, with the x-axis representing the epochs and the y-axis representing the loss values. The plot provides insights into the performance of the model over the training epochs, showing how the loss values change over time.

# Testing Data

In [ ]:
# These lines calculate the predictions (y_hat) using the trained model on the test data (x_test). 
# Then, the inverse transform is applied to y_hat and y_test using the min_max_scaler to obtain the actual
# values in their original scale. The predicted values (y_hat_inversed) and actual 
# values (y_test_inversed) are converted to flat list
Results_test = {}
y_hat = model.predict(x_test)
y_hat_inversed = min_max_scaler.inverse_transform(y_hat.reshape(-1,1)).flatten().tolist()
y_test_inversed = min_max_scaler.inverse_transform(y_test.reshape(-1,1)).flatten().tolist()

# These lines create a dictionary Results_test to store the predicted values, actual values, and index values.
# The predicted and actual values are assigned to the corresponding keys, and the index values are generated using np.arange()
Results_test['y_hat'] =y_hat_inversed
Results_test['y_test'] =y_test_inversed
Results_test['Index'] = np.arange(0, len(y_test_inversed))

# These lines create a DataFrame results_df_test from the Results_test dictionary. The DataFrame is then displayed.
results_df_test = pd.DataFrame.from_dict(Results_test)
display(results_df_test)


# Plotting
fig = go.Figure()
fig.add_trace(go.Scatter(x=results_df_test['Index'], y=results_df_test['y_hat'], name='Predicted',mode='markers+lines'))
fig.add_trace(go.Scatter(x=results_df_test['Index'], y=results_df_test['y_test'], name='Actual' ,mode='markers+lines'))
fig.update_layout(
            xaxis_title='Index',
            yaxis_title='Value',
            title='Tesing Data Predicted vs Actual Curves'
        )
fig.write_html('plots/LSTM-predicted_actual_curves.html')

fig.show()

# Summary:
# The code calculates the predictions of the LSTM model on the test data and transforms them back to their original scale.
# It creates a DataFrame to store the predicted and actual values along with their corresponding indices. 
# Then, it generates a plot using Plotly with separate lines for the predicted and actual values.
# The plot is saved as an HTML file and displayed. 
# This allows visual comparison between the predicted and actual values of the testing data.

In [ ]:

Results_train = {}

# for Training data 
y_train_hat = model.predict(x_train)
y_train_hat_inversed = min_max_scaler.inverse_transform(y_train_hat.reshape(-1,1)).flatten().tolist()
y_train_inversed = min_max_scaler.inverse_transform(y_train.reshape(-1,1)).flatten().tolist()

Results_train['y_train_hat'] =y_train_hat_inversed
Results_train['y_train'] =y_train_inversed
Results_train['Index'] = np.arange(0, len(y_train_inversed))
results_df_train = pd.DataFrame.from_dict(Results_train)
display(results_df_train)



# Plotting
fig = go.Figure()
fig.add_trace(go.Scatter(x=results_df_train['Index'], y=results_df_train['y_train_hat'], name='Predicted',mode='markers+lines'))
fig.add_trace(go.Scatter(x=results_df_train['Index'], y=results_df_train['y_train'], name='Actual' ,mode='markers+lines'))
fig.update_layout(
            xaxis_title='Index',
            yaxis_title='Value',
            title='Tesing Data Predicted vs Actual Curves'
        )
fig.write_html('plots/LSTM-predicted_actual_curves.html')
fig.show()

# Summary:
# The code calculates the predictions of the LSTM model on the training data and transforms them back to their original scale. 
# It creates a DataFrame to store the predicted and actual values along with their corresponding indices 
# for the training data. Then, it generates a plot using Plotly with separate lines for the predicted and actual values. 
# The plot is saved as an HTML file and displayed. This allows visual comparison between the predicted and actual 
# values of the training data.

In [ ]:
results_df_test

In [ ]:
import numpy as np

def calculate_errors(predicted, actual):
    # Convert lists to numpy arrays
    predicted = np.array(predicted)
    actual = np.array(actual)

    # Mean Absolute Error (MAE)
    mae = np.mean(np.abs(predicted - actual))

    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(np.mean((predicted - actual) ** 2))

    # Mean Squared Error (MSE)
    mse = np.mean((predicted - actual) ** 2)

    # Mean Absolute Percentage Error (MAPE)
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100

    return mae, rmse, mse, mape
train_mae , train_rmse , train_mse,train_mape = calculate_errors(results_df_train['y_train_hat'],results_df_train['y_train'])
test_mae , test_rmse , test_mse,test_mape = calculate_errors(results_df_test['y_hat'],results_df_test['y_test'])

print('Results on Training Data')
print(f'MAE : {train_mae}\nRMSE : {train_rmse}\nMSE : {train_mse}\nMAPE : {train_mape}'.format())
print('\n')
print('Results on Testing Data')
print(f'MAE : {test_mae}\nRMSE : {test_rmse}\nMSE : {test_mse}\nMAPE : {test_mape}')

# Summary:
# The code defines a function to calculate various error metrics (MAE, RMSE, MSE, MAPE) given predicted and actual values.
# Then, the function is called to calculate the error metrics for both the training and testing data. 
# The calculated error metrics provide a quantitative assessment of the model's performance in terms of the prediction accuracy.

In [ ]:
import pandas as pd

# Read the existing error history DataFrame from CSV
error_history = pd.read_csv('errorDatabaseHistory.csv')

# Create a new record as a DataFrame
new_record = pd.DataFrame({
    'modelName': [model_name],
    'modelConfig': [model.get_config()],
    'train_rmse': [train_rmse],
    'train_mae': [train_mae],
    'train_mse': [train_mse],
    'train_mape': [train_mape],
    'test_rmse': [test_rmse],
    'test_mae': [test_mae],
    'test_mse': [test_mse],
    'test_mape': [test_mape]
})

# Append the new record to the existing error history DataFrame
error_history = pd.concat([error_history, new_record], ignore_index=True)

# Save the updated error history DataFrame to CSV
error_history.to_csv('errorDatabaseHistory.csv', index=False)


# Final

In [ ]:
error_history.iloc[7][0]